In [ ]:
#!pip install shap
#!pip install sklearn
#!pip install astunparse
import shap
import numpy as np
from sklearn.metrics import mean_squared_error
from math import sqrt
import tensorflow as tf
tf.compat.v1.disable_v2_behavior()


In [ ]:
#qualifying here.... load model and predict
from tensorflow.keras.models import load_model
import tensorflow as tf
import pandas as pd
from numpy import array
from numpy import asarray
import numpy as np
from matplotlib import *
import matplotlib.pyplot as plt
from numpy import concatenate
print(tf.__version__)
#fix with tf 2.1
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)




#LSTM 16- 1 layer - timestep =1
import tensorflow as tf
import numpy as np
from numpy import array
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder    

# source https://machinelearningmastery.com/convert-time-series-supervised-learning-problem-python/
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequences)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out-1
        # check if we are beyond the dataset
        if out_end_ix > len(sequences):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1:out_end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

def Fill_data(df,mean,timesteps,n_features):
    columns = df.columns
    row_stand = df.iloc[0]
    for i in range (1,timesteps):
        row_temp = np.array([])
        for j in range(0,4):
            temp = row_stand[j]-i
            row_temp = np.append(row_temp,temp)
        for j in range(4,n_features):
            row_temp = np.append(row_temp,mean[j])

        minus = 0
        if(row_temp[0]<0): #index
            row_temp[0] = 0
        if(row_temp[1]<0): #hour
            row_temp[1] += 24
            minus = 1
        if(row_temp[2]<0 or minus == 1): #day of the year
            row_temp[2] += 365
            if(minus == 1):
                row_temp[2] -= 1
                minus= 0
        if(row_temp[3]<0): #weekday : from 0 to 6
            row_temp[4] = 0
        row_temp= pd.DataFrame(row_temp.reshape(1, len(row_temp)),columns=columns)
        df = pd.concat([row_temp,df],ignore_index=True,axis=0)
    return df

def Evaluation(my_model,X_test,y_real,scaler,timesteps,n_features):
    from numpy import concatenate
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    y_pred = my_model.predict(X_test)

    X_test = X_test.reshape((X_test.shape[0], timesteps*(n_features-1)))
    #predict values
    from numpy import concatenate
    X_test = X_test[:, -(n_features-1):]
    y_pred = concatenate((X_test,y_pred), axis=1)
    y_pred = scaler.inverse_transform(y_pred)
    y_pred = y_pred[:,-1]

    y_real = concatenate((X_test,y_real), axis=1)
    y_real = scaler.inverse_transform(y_real)
    y_real = y_real[:,-1]

    RMSE = sqrt(mean_squared_error(y_real, y_pred))
    from sklearn.metrics import r2_score
    R2 = r2_score(y_real, y_pred)
    return RMSE,R2

def Evaluation2(my_model,X_test,y_real,scaler,timesteps,n_features):
    from numpy import concatenate
    from sklearn.metrics import mean_squared_error
    from math import sqrt
    y_pred = my_model.predict(X_test)

    X_test = X_test.reshape((X_test.shape[0], timesteps*(n_features-1)))
    #predict values
    from numpy import concatenate
    X_test = X_test[:, -(n_features-1):]
    y_pred = concatenate((X_test,y_pred), axis=1)
    y_pred = scaler.inverse_transform(y_pred)
    y_pred = y_pred[:,-1]

    return y_pred

def Quatifying(nodes,timesteps,data_path,Train_file,Test_file, Model_name,y_name,Output_file,original_data):

    #load model from file - the best model
    loaded_model = load_model(Model_name)

    #compile the model
    #opt = tf.keras.optimizers.Adam(learning_rate=0.05)
    #loaded_model.compile(loss = 'mse',
    #     optimizer = opt, 
    #     metrics = ['mean_squared_error'])

    #Load X_test, y_real
    data = pd.read_csv(data_path) #four year data
    data = data.dropna()
    Train1 = pd.read_csv(Train_file)
    Train1 = Train1.dropna()
    Test1 = pd.read_csv(Test_file)
    Test1 = Test1.dropna()
    
    New_data = [Train1, Test1]
    New_data = pd.concat(New_data)
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    New_data = scaler.fit_transform(New_data)

    Train =New_data[0:Train1.shape[0],:]
    Test = New_data[Train1.shape[0]:,:]
    
    Quatify = data[data['Index']>35424 ] #get data from 15/1/2020

    n_features=np.shape(Train1)[1] #12 inluding output

    columns = Train1.columns
    mean = Train1.mean(axis=0)

    #padding because of timesteps
    Quatifying = Fill_data(Quatify,mean,timesteps, n_features )

    #tranform train and test.
    New_data = Quatifying.values

    New_data = scaler.transform(New_data)

    X_Quatifying, y_real = split_sequences(New_data, timesteps, 1)

    print( 'size after appling timesteps: ',X_Quatifying.shape,y_real.shape)

    #Evaluate
    y_pred = Evaluation2(loaded_model,X_Quatifying,y_real,scaler,timesteps,n_features)
    
    df = pd.DataFrame(Quatify)
    cols = list(Quatify.columns)
    cols = [cols[0], cols[-1]]
    df3 = df[cols]
    columnName = df3.columns

    

    df3['Predicted']= y_pred


    data_original = pd.read_excel(original_data, 'Sheet1')
    data_original= data_original[data_original['FECHA_UNIDAD']>='2016-01-01 00:00:00']
    Features = data_original.iloc[:,6:]
    FECHA_UNIDAD =  data_original[['FECHA_UNIDAD']]

    FECHA_UNIDAD.insert(0, 'Index', np.arange(1,len(FECHA_UNIDAD[:])+1))

    Target = data_original[[y_name]]
    Target = pd.to_numeric(Target[y_name], errors='coerce')
    Target_data = pd.concat([FECHA_UNIDAD,Features,Target], axis=1)
    Target_data = Target_data.rename(columns={'FECHA_UNIDAD': 'DateTime'})
    Target_data = Target_data.dropna()
    Target_data = Target_data[Target_data['DateTime']> '2020-01-15 23:00:00']
    Target_data['Predicted']= df3['Predicted'].values

    Target_data.to_csv(Output_file,index=False)

    print('\nDone wtire to file...',Output_file,'\n')
    #return Quatify,y_pred #dataframe include X, and predict value.
    
def Preprocess(nodes,timesteps,data_path,Train_file,Test_file, Model_name,y_name,Output_file,original_data):

    #load model from file - the best model
    loaded_model = load_model(Model_name)

    #Load X_test, y_real
    data = pd.read_csv(data_path) #four year data
    data = data.dropna()
    Train1 = pd.read_csv(Train_file)
    Train1 = Train1.dropna()
    Test1 = pd.read_csv(Test_file)
    Test1 = Test1.dropna()
    
    New_data = [Train1, Test1]
    New_data = pd.concat(New_data)
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    New_data = scaler.fit_transform(New_data)

    Train =New_data[0:Train1.shape[0],:]
    Test = New_data[Train1.shape[0]:,:]
    
    Training = Train1
    

    n_features=np.shape(Train1)[1] #12 inluding output

    columns = Train1.columns
    mean = Train1.mean(axis=0)

    #padding because of timesteps
    Training = Fill_data(Training,mean,timesteps, n_features )
    Testing = Fill_data(Test1,mean,timesteps, n_features )

    #tranform train and test.
    New_data = Training.values
    Testing  = Testing.values

    New_data = scaler.transform(New_data)
    New_test =  scaler.transform(Testing)

    X_Training, y_training = split_sequences(New_data, timesteps, 1)
    X_Testing, y_testing = split_sequences(New_test, timesteps, 1)

    print( 'size after appling timesteps: ',X_Training.shape,y_training.shape)
    print ('done')
    return loaded_model, X_Training, y_training, X_Testing, y_testing, scaler
    
    
    
    


In [ ]:
#!pip install matplotlib

In [ ]:
#CO_mg/m3
#SO2_ug/m3
#PM2.5_ug/m3
#O3_ug/m3
nodes=64
timesteps=3
data_path = 'C:\\data\\4_years_data\\O3.csv'
Model_name = 'C:\\data\\best_model\\O3_best.h5'
Train_file = 'C:\\data\\Data_paper\\O3_Train.csv'
Test_file = 'C:\\data\\Data_paper\\O3_Test.csv'
original_data = 'C:\\data\\Data_belisario2.xlsx'
#Output_file = 'PM2_5_Results_LSTM_96_12_Quatifying.csv'
y_name = 'O3_mg/m3'
Output_file = 'C:\\data\\SHAP\\O3_Results_Quatifying.csv'

loaded_model, X_Training, y_training, X_Testing, y_testing, scaler = Preprocess(nodes,timesteps,data_path,Train_file,Test_file, Model_name,y_name,Output_file,original_data)



In [ ]:
temp = loaded_model.predict(X_Training)
print(np.shape(temp))
print(np.shape(X_Training))


In [ ]:
import shap
import tensorflow as tf
print (shap.__version__)
print (tf.__version__)
loaded_model.summary()

In [ ]:
#https://medium.com/datadriveninvestor/time-step-wise-feature-importance-in-deep-learning-using-shap-e1c46a655455
#simpler NN https://christophm.github.io/interpretable-ml-book/neural-networks.html 
#this is under development.
#error: https://stackoverflow.com/questions/64401570/error-using-shap-with-simplernn-sequential-model
import shap
import tensorflow.keras.backend 
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import load_model
import shap

tf.compat.v1.disable_eager_execution()
#model = load_model(Model_name)

#pred_x = regressor.predict_classes(X_Training)

#shap.explainers.deep.deep_tf.op_handlers["AddV2"] = shap.explainers.deep.deep_tf.passthrough
#explainer  = shap.DeepExplainer((regressor.layers[0].input, regressor.layers[-1].output),data)
#background = X_Training[np.random.choice(X_Training.shape[0], 200, replace=False)]

#from shap.explainers.deep.deep_tf import passthrough
#shap.explainers._deep.deep_tf.op_handlers['AddV2'] = passthrough
#explainer  = shap.DeepExplainer(regressor,background)

background = X_Training[np.random.choice(X_Training.shape[0], 15000, replace=False)]
#test = X_Training[np.random.choice(X_Training.shape[0], 300, replace=False)]
explainer = shap.DeepExplainer(loaded_model, background)
shap_vals = explainer.shap_values(X_Testing)
f_names = ['Index','Hours','Julian_day','Week_day','Temperature','SR','p','Prec','RH','WD','WS']
shap.summary_plot(shap_vals[0][:, timesteps-1, :],feature_names=f_names,  plot_type="bar")



